In [0]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('Spark') \
                    .getOrCreate()

In [0]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)

root
-- employee_name: string (nullable = true)
-- department: string (nullable = true)
-- state: string (nullable = true)
-- salary: long (nullable = true)
-- age: long (nullable = true)
-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
James |Sales |NY |90000 |34 |10000|
Michael |Sales |NY |86000 |56 |20000|
Robert |Sales |CA |81000 |30 |23000|
Maria |Finance |CA |90000 |24 |23000|
Raman |Finance |CA |99000 |40 |24000|
Scott |Finance |NY |83000 |36 |19000|
Jen |Finance |NY |79000 |53 |15000|
Jeff |Marketing |CA |80000 |25 |18000|
Kumar |Marketing |NY |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+

In [0]:
df.groupBy("department").sum("salary").show(truncate=False)


+----------+-----------+
department|sum(salary)|
+----------+-----------+
Sales |257000 |
Finance |351000 |
Marketing |171000 |
+----------+-----------+

In [0]:
df.groupBy("department").count().show(truncate=False)


+----------+-----+
department|count|
+----------+-----+
Sales |3 |
Finance |4 |
Marketing |2 |
+----------+-----+

In [0]:
df.groupBy("department").min("salary").show(truncate=False)


+----------+-----------+
department|min(salary)|
+----------+-----------+
Sales |81000 |
Finance |79000 |
Marketing |80000 |
+----------+-----------+

In [0]:
df.groupBy("department").max("salary").show(truncate=False)


+----------+-----------+
department|max(salary)|
+----------+-----------+
Sales |90000 |
Finance |99000 |
Marketing |91000 |
+----------+-----------+

In [0]:
df.groupBy("department").avg( "salary").show(truncate=False)


+----------+-----------------+
department|avg(salary) |
+----------+-----------------+
Sales |85666.66666666667|
Finance |87750.0 |
Marketing |85500.0 |
+----------+-----------------+

In [0]:
df.groupBy("department").mean( "salary").show(truncate=False)


+----------+-----------------+
department|avg(salary) |
+----------+-----------------+
Sales |85666.66666666667|
Finance |87750.0 |
Marketing |85500.0 |
+----------+-----------------+

In [0]:
#GroupBy on multiple columns
df.groupBy("department","state") \
    .sum("salary","bonus") \
    .show(truncate=False)

+----------+-----+-----------+----------+
department|state|sum(salary)|sum(bonus)|
+----------+-----+-----------+----------+
Sales |NY |176000 |30000 |
Sales |CA |81000 |23000 |
Finance |CA |189000 |47000 |
Finance |NY |162000 |34000 |
Marketing |NY |91000 |21000 |
Marketing |CA |80000 |18000 |
+----------+-----+-----------+----------+

In [0]:
from pyspark.sql.functions import sum,avg,max,min,mean,count
df.groupBy("department") \
    .agg(sum("salary").alias("sum_salary"), \
         avg("salary").alias("avg_salary"), \
         sum("bonus").alias("sum_bonus"), \
         max("bonus").alias("max_bonus") \
     ) \
    .show(truncate=False)

+----------+----------+-----------------+---------+---------+
department|sum_salary|avg_salary |sum_bonus|max_bonus|
+----------+----------+-----------------+---------+---------+
Sales |257000 |85666.66666666667|53000 |23000 |
Finance |351000 |87750.0 |81000 |24000 |
Marketing |171000 |85500.0 |39000 |21000 |
+----------+----------+-----------------+---------+---------+

In [0]:
from pyspark.sql.functions import col
df.groupBy("department") \
    .agg(sum("salary").alias("sum_salary"), \
      avg("salary").alias("avg_salary"), \
      sum("bonus").alias("sum_bonus"), \
      max("bonus").alias("max_bonus")) \
    .where(col("sum_bonus") >= 50000) \
    .show(truncate=False)

+----------+----------+-----------------+---------+---------+
department|sum_salary|avg_salary |sum_bonus|max_bonus|
+----------+----------+-----------------+---------+---------+
Sales |257000 |85666.66666666667|53000 |23000 |
Finance |351000 |87750.0 |81000 |24000 |
+----------+----------+-----------------+---------+---------+